In [16]:
import pandas as pd

In [17]:
column_template = ['ts',
              'uid',
              'id.orig_h',
              'id.orig_p',
              'id.resp_h',
              'id.resp_p',
              'proto',
              'service',
              'duration',
              'orig_bytes',
              'resp_bytes',
              'conn_state',
              'local_orig',
              'local_resp',
              'missed_bytes',
              'history',
              'orig_pkts',
              'orig_ip_bytes',
              'resp_pkts',
              'resp_ip_bytes',
              'label']

In [18]:
capture_44 = "./data/IoTScenarios/CTU-IoT-Malware-Capture-44-1/bro/conn.log.labeled"
df44 = pd.read_table(filepath_or_buffer=capture_44, skiprows=8, header=None) #, nrows=100000)
df44.columns= column_template
df44.drop(df44.tail(1).index,inplace=True)

In [19]:
capture_49 = "./data/IoTScenarios/CTU-IoT-Malware-Capture-49-1/bro/conn.log.labeled"
df49 = pd.read_table(filepath_or_buffer=capture_49, skiprows=8, header=None) #, nrows=100000)
df49.columns= column_template
df49.drop(df49.tail(1).index,inplace=True)

C:\Users\hullc\AppData\Local\Temp\ipykernel_16660\2152934637.py:2: DtypeWarning: Columns (0,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df49 = pd.read_table(filepath_or_buffer=capture_49, skiprows=8, header=None) #, nrows=100000)


In [20]:
#frames=[df44]
frames= [df44, df49]
#names = ["df1", "df20", "df21", "df3", "df34", "df35", "df36", "df42", "df44", "df48", "df49", "df60", "df7", "df8", "df9"]

0 = Benign
1 = C&C
2 = C&C-FileDownload
3 = DDoS
4 = FileDownload
5 = PartOfAHorizontalPortScan


In [21]:
for frame in frames:
  frame.loc[(frame.label == '-   Malicious   PartOfAHorizontalPortScan'), 'label'] = '5'
  frame.loc[(frame.label == '(empty)   Malicious   PartOfAHorizontalPortScan'), 'label'] = '5'
  frame.loc[(frame.label == '-   Malicious   Okiru'), 'label'] = 'Okiru'
  frame.loc[(frame.label == '(empty)   Malicious   Okiru'), 'label'] = 'Okiru'
  frame.loc[(frame.label == '-   Benign   -'), 'label'] = '0'
  frame.loc[(frame.label == '(empty)   Benign   -'), 'label'] = '0'
  frame.loc[(frame.label == '-   Malicious   DDoS'), 'label'] = '3'
  frame.loc[(frame.label == '-   Malicious   C&C'), 'label'] = '1'
  frame.loc[(frame.label == '(empty)   Malicious   C&C'), 'label'] = '1'
  frame.loc[(frame.label == '-   Malicious   Attack'), 'label'] = 'Attack'
  frame.loc[(frame.label == '(empty)   Malicious   Attack'), 'label'] = 'Attack'
  frame.loc[(frame.label == '-   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
  frame.loc[(frame.label == '(empty)   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
  frame.loc[(frame.label == '-   Malicious   C&C-FileDownload'), 'label'] = '2'
  frame.loc[(frame.label == '-   Malicious   C&C-Torii'), 'label'] = 'C&C-Torii'
  frame.loc[(frame.label == '-   Malicious   C&C-HeartBeat-FileDownload'), 'label'] = 'C&C-HeartBeat-FileDownload'
  frame.loc[(frame.label == '-   Malicious   FileDownload'), 'label'] = '4'
  frame.loc[(frame.label == '-   Malicious   C&C-Mirai'), 'label'] = 'C&C-Mirai'
  frame.loc[(frame.label == '-   Malicious   Okiru-Attack'), 'label'] = 'Okiru-Attack'

In [22]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [23]:
for frame in frames:
  frame = frame.drop(columns=['ts','uid','id.orig_h','id.orig_p','id.resp_h','id.resp_p',
                               'service','local_orig','local_resp','history'], inplace=True)

In [24]:
for frame in frames:
  frame['duration'].replace('-','0',inplace=True)
  frame['orig_bytes'].replace('-','0',inplace=True)
  frame['resp_bytes'].replace('-','0',inplace=True)
  frame.fillna(-1,inplace=True)

In [25]:
df_c = pd.concat(frames)
df_c = pd.get_dummies(df_c, columns=['proto'])
df_c = pd.get_dummies(df_c, columns=['conn_state'])
df_c.sort_values(by=['label'], inplace=True)

In [26]:
df0 = df_c[df_c['label'] == '0']
df1 = df_c[df_c['label'] == '1']
df2 = df_c[df_c['label'] == '2']
df3 = df_c[df_c['label'] == '3']
df4 = df_c[df_c['label'] == '4']
df5 = df_c[df_c['label'] == '5'].iloc[::1859,:]

In [27]:
print(df0.shape)
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(3876, 20)
(1936, 20)
(12, 20)
(1, 20)
(14, 20)
(2908, 20)


In [28]:
df_f = pd.concat([df0, df1, df2, df3,df4,df5])

cols = df_f.columns.tolist()
newOrder = cols[:7] + cols[9:]
newOrder.append(cols[8])

In [29]:
df_f.to_csv('./data/preprocessed/bigframeMirai2.csv', columns=newOrder)
df_f.to_csv('./data/preprocessed/bigframeMirai2_TT.csv', columns=['label'])
